## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en analysant la rentabilité brute dans les **principales villes étudiantes françaises** (studios et T1 ≤45m²)

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🔍 Besoins data de Léa
- Évaluer le **taux de vacance locative** en France pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Visualiser les **villes à forte concentration étudiante** en France
- Analyser l'**évolution du prix au m² à l'achat et des loyers étudiants** en France
- Etudier **la rentabilité moyenne en France** en 2024
- Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux)
- Comparer **les quartiers les plus rentables (rentabilité brute)** à ?
- Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?
- Analyser la **localisation des universités/grandes écoles** à ?
- Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? 
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Analyser la **rentabilité locative brute** pour appartements étudiants ≤45m² dans **23 grandes villes françaises**
2. Explorer visuellement les **villes à forte concentration étudiante** et analyser les **taux de vacance locative**
3. Obtenir un **classement des villes** par rentabilité, prix et loyers pour décider où investir avec 200k€

## 📚 Import des bibliothèques ##

In [26]:
import pandas as pd 
import plotly.express as px
import geopandas as gpd
import requests
import os
import osmnx as ox
from shapely.geometry import Point

# 2e vision : Analyse resserrée sur des villes #

### Analyser la **dynamique du marché immobilier local : croissance ou baisse des prix et loyers sur les 5 dernières années** (entre Rennes et Bordeaux) ###

In [ ]:
print("Partie Axel")

### Comparer **les quartiers les plus rentables (rentabilité brute)** à ? ###

In [ ]:
print("Partie Lucien")

### Analyser la **localisation/nombre des transports en commun** pour identifier les zones les plus attractives pour les étudiants à ?  ###

In [ ]:
print("Partie Valentine")

### Analyser la **localisation des universités/grandes écoles** à ? ###

In [3]:
url_df_enseignement_sup = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits-detail_etablissements.csv"

df_enseignement_sup = pd.read_csv(url_df_enseignement_sup, delimiter=';')

df_enseignement_sup.head()

,Rentrée universitaire,catégorie d'établissement,secteur d'établissement,code UAI de l'établissement,sigle de l'établissement,libellé de l'établissement,libellé complémentaire de l'établissement,Code UAI de la composante,sigle de la composante,libellé de la composante,...,id unité urbaine,unité urbaine,code commune,Commune,gps,degetu,degre_etudes,nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE,dont femmes,dont hommes
0,2023,"Écoles de commerce, gestion et vente",Privé,0562044T,EC TECH SUP PR,GROUPE AFTEC,IPAC BACHELOR FACTORY VANNES,0562044T,EC TECH SUP PR,GROUPE AFTEC,...,UU56501,Vannes,56158,Plescop,"47.67764394859038, -2.80211043128016",3.0,BAC + 3,112,96,16
1,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,0060656F,E.COM,ECOLE DE COMMERCE,...,UU06701,Nice,06152,Valbonne,"43.61328883276337, 7.056290282560445",6.0,BAC + 6 et plus,19,9,10
2,2023,"Écoles de commerce, gestion et vente",Privé,0590346F,SKEMA LILLE,SKEMA BUSINESS SCHOOL LILLE,NaN,216P0001,SKEMA CHINE,SKEMA BUSINESS SCHOOL,...,CHI99216,Suzhou,99216,Suzhou,NaN,4.0,BAC + 4,389,67,322
3,2023,"Écoles de commerce, gestion et vente",Privé,0590350K,EDHEC LILLE,EDHEC BUSINESS SCHOOL,MEMBRE FUPL,0755719J,EDHEC,CENTRE DE FORMATION CONTINUE,...,UU00851,Paris,75102,Paris 2e,NaN,5.0,BAC + 5,387,162,225
4,2023,"Écoles de commerce, gestion et vente",Privé,0593202K,IESEG LILLE,INST ECO SCIENT GESTION LILLE,MEMBRE FUPL,0593202K,IESEG LILLE,INST ECO SCIENT GESTION LILLE,...,UU59702,Lille,59350,Lille,"50.63553116922688, 3.044909976114497",2.0,BAC + 2,739,285,454


In [12]:
df_rennes = df_enseignement_sup[df_enseignement_sup['Commune'] == 'Rennes']

print(f"Nombre de lignes à Rennes :", df_rennes.shape[0])
print(f"Nombres de lignes avec gps à Rennes :", (df_rennes['gps'].isnull() == False).sum())
print(f"Nombres de lignes sans gps à Rennes avant drop:", df_rennes['gps'].isnull().sum())

df_rennes = df_rennes.dropna(subset=['gps'])

print(f"Nombres de lignes sans gps à Rennes après drop:", df_rennes['gps'].isnull().sum())
df_rennes.head()

Nombre de lignes à Rennes : 1531
Nombres de lignes avec gps à Rennes : 1461
Nombres de lignes sans gps à Rennes avant drop: 70
Nombres de lignes sans gps à Rennes après drop: 0


,Rentrée universitaire,catégorie d'établissement,secteur d'établissement,code UAI de l'établissement,sigle de l'établissement,libellé de l'établissement,libellé complémentaire de l'établissement,Code UAI de la composante,sigle de la composante,libellé de la composante,...,id unité urbaine,unité urbaine,code commune,Commune,gps,degetu,degre_etudes,nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE,dont femmes,dont hommes
322,2022,Lycées,Privé,0352267Z,ECP PR,ECOLE PRIVEE COIF ESTHETIQUE,L'ACADEMY - SILVYA TERRADE REN,0352267Z,ECP PR,ECOLE PRIVEE COIF ESTHETIQUE,...,UU35701,Rennes,35238,Rennes,"48.10427073898263, -1.6691603554365282",2.0,BAC + 2,9,8,1
583,2023,Universités,Public,0353074B,UR (EPE),UNIVERSITE,DE RENNES (EPE),0352076S,UFR SC VIE ENV,UFR SCIENCES VIE ENVIRONNEMENT,...,UU35701,Rennes,35238,Rennes,"48.11412271889446, -1.6376903820335271",6.0,BAC + 6 et plus,61,35,26
820,2021,Écoles d'ingénieurs,Public,0350097R,INSA RENNES,INST NAT SC APPLIQ RENNES,NaN,0350097R,INSA RENNES,INST NAT SC APPLIQ RENNES,...,UU35701,Rennes,35238,Rennes,"48.1215800418907, -1.6329696980490491",4.0,BAC + 4,411,126,285
878,2019,Universités,Public,0290346U,U BREST,UNIVERSITE BREST,BRETAGNE OCCIDENTALE,0352076S,UFR SC VIE ENV,UFR SCIENCES VIE ENVIRONNEMENT,...,UU35701,Rennes,35238,Rennes,"48.11412271889446, -1.6376903820335271",NaN,NaN,48,27,21
1196,2015,Universités,Public,0350936C,U RENNES 1,UNIVERSITE RENNES 1,UNIVERSITE RENNES 1,0350075S,IG RENNES 1,INSTITUT DE GESTION RENNES 1,...,UU35701,Rennes,35238,Rennes,"48.11728104154247, -1.6723406266225596",NaN,NaN,1339,753,586


In [16]:
# Nettoyage des coordonnées
df_rennes[['lat', 'lon']] = df_rennes['gps'].str.split(',', expand=True)
df_rennes['lat'] = df_rennes['lat'].astype(float)
df_rennes['lon'] = df_rennes['lon'].astype(float)

print(f"Nombre de lignes à Rennes :", df_rennes[['lat', 'lon']].shape[0])
print(f"Nombres de lignes avec lat et lon à Rennes :", ((df_rennes['lat'].isnull() == False) & (df_rennes['lon'].isnull() == False)).sum())
print(f"Nombres de lignes sans lat et lon à Rennes avant drop:", ((df_rennes['lat'].isnull()) & (df_rennes['lon'].isnull())).sum())

# On enlève les lignes sans coordonnées
df_rennes = df_rennes.dropna(subset=['lat', 'lon'])

print(f"Nombres de lignes sans lat et lon à Rennes après drop:", ((df_rennes['lat'].isnull()) & (df_rennes['lon'].isnull())).sum())

Nombre de lignes à Rennes : 1461
Nombres de lignes avec lat et lon à Rennes : 1461
Nombres de lignes sans lat et lon à Rennes avant drop: 0
Nombres de lignes sans lat et lon à Rennes après drop: 0


In [ ]:
# Transformation en GeoDataFrame 
df_rennes = gpd.GeoDataFrame(
    df_rennes,
    geometry=gpd.points_from_xy(df_rennes.lon, df_rennes.lat),
    crs="EPSG:4326"  # WGS84
)

In [17]:
url_iris = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/contours-iris-pe.gpkg"
local_path = "contours-iris-pe.gpkg"

# Télécharger une seule fois
if not os.path.exists(local_path):
    print("Téléchargement du fichier...")
    r = requests.get(url_iris)
    with open(local_path, "wb") as f:
        f.write(r.content)

# Lister les couches avec fiona
from fiona import listlayers
layers = listlayers(local_path)
print("Couches disponibles:", layers)

# Charger la couche principale
iris = gpd.read_file(local_path, layer=layers[0])
iris.head()


Téléchargement du fichier...
Couches disponibles: ['contours_iris_pe']


,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry
0,IRIS____0000000670430101,67043,Bischheim,0101,670430101,Bischheim,A,"MULTIPOLYGON (((1052052.5 6848302.2, 1052346.7..."
1,IRIS____0000000132020101,13202,Marseille 2e Arrondissement,0101,132020101,Marseille 2e Arrondissement,A,"MULTIPOLYGON (((892392.4 6249040.5, 892189 624..."
2,IRIS____0000000561850101,56185,Quéven,0101,561850101,Quéven,A,"MULTIPOLYGON (((217035.9 6759952.7, 216941.9 6..."
3,IRIS____0000000930630101,93063,Romainville,0101,930630101,Romainville,A,"MULTIPOLYGON (((659020 6865990.2, 658978.6 686..."
4,IRIS____0000000940480101,94048,Marolles-en-Brie,0101,940480101,Marolles-en-Brie,A,"MULTIPOLYGON (((667454 6850589.8, 667714.1 685..."


In [18]:
url_ref_iris = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/reference_IRIS_geo2025.xlsx"

iris_noms = pd.read_excel(url_ref_iris)

iris_noms = iris_noms.rename(columns={'CODE_IRIS': 'code_iris'})

# fusion avec ton GeoDataFrame (qui contient les codes IRIS)
iris = iris.merge(iris_noms[['code_iris', 'LIB_IRIS', 'LIBCOM']], on='code_iris', how='left')

iris_noms.head()


,code_iris,LIB_IRIS,TYP_IRIS,GRD_QUART,DEPCOM,LIBCOM,UU2020,REG,DEP
0,010010000,L'Abergement-Clémenciat (commune non irisée),Z,0100100,01001,L'Abergement-Clémenciat,01000,84.0,01
1,010020000,L'Abergement-de-Varey (commune non irisée),Z,0100200,01002,L'Abergement-de-Varey,01000,84.0,01
2,010040101,Les Pérouses-Triangle d'Activités,H,0100401,01004,Ambérieu-en-Bugey,01303,84.0,01
3,010040102,Longeray-Gare,H,0100401,01004,Ambérieu-en-Bugey,01303,84.0,01
4,010040201,Centre-Saint-Germain-Vareilles,H,0100402,01004,Ambérieu-en-Bugey,01303,84.0,01


In [33]:
# Filtrer
iris_rennes = iris[iris['nom_commune'].str.contains("Rennes", case=False, na=False)].copy()

print(f"Rennes contient", len(iris_rennes), "IRIS")

iris_rennes.head()

Rennes contient 146 IRIS


,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,geometry,LIB_IRIS,LIBCOM
489,IRIS____0000000352380308,35238,Rennes,0308,352380308,Rennes,A,"MULTIPOLYGON (((349041.8 6789540.4, 349108.7 6...",Z.A. Ouest,Rennes
516,IRIS____0000000352380403,35238,Rennes,0403,352380403,Rennes,A,"MULTIPOLYGON (((352060.5 6791899.6, 351994.9 6...",Z.A. Nord,Rennes
575,IRIS____0000000352380607,35238,Rennes,0607,352380607,Rennes,A,"MULTIPOLYGON (((355296.3 6788725.3, 354873.5 6...",Plaine de Baud,Rennes
597,IRIS____0000000352380704,35238,Rennes,0704,352380704,Rennes,A,"MULTIPOLYGON (((354457.9 6788601.8, 354873.5 6...",Z.A. Sud Est,Rennes
1032,IRIS____0000000352380307,35238,Rennes,0307,352380307,Rennes,D,"MULTIPOLYGON (((348304.3 6789477.5, 348125.4 6...",Champeaux,Rennes


In [35]:
# On met tout dans la même CRS projetée (ex: EPSG:2154 ou utiliser EPSG:4326 par la suite)
iris_rennes = iris_rennes.to_crs(epsg=4326)
df_rennes = df_rennes.to_crs(epsg=4326)


In [36]:
# spatial join : chaque établissement rattaché à un IRIS
etabs_par_iris_rennes = gpd.sjoin(df_rennes, iris_rennes, how="inner", predicate="within")
etabs_par_iris_rennes.head()

,Rentrée universitaire,catégorie d'établissement,secteur d'établissement,code UAI de l'établissement,sigle de l'établissement,libellé de l'établissement,libellé complémentaire de l'établissement,Code UAI de la composante,sigle de la composante,libellé de la composante,...,index_right,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,LIB_IRIS,LIBCOM
322,2022,Lycées,Privé,0352267Z,ECP PR,ECOLE PRIVEE COIF ESTHETIQUE,L'ACADEMY - SILVYA TERRADE REN,0352267Z,ECP PR,ECOLE PRIVEE COIF ESTHETIQUE,...,10225,IRIS____0000000352380207,35238,Rennes,0207,352380207,Rennes,H,Saint-Hélier,Rennes
583,2023,Universités,Public,0353074B,UR (EPE),UNIVERSITE,DE RENNES (EPE),0352076S,UFR SC VIE ENV,UFR SCIENCES VIE ENVIRONNEMENT,...,12606,IRIS____0000000352380605,35238,Rennes,0605,352380605,Rennes,H,Campus de Beaulieu,Rennes
820,2021,Écoles d'ingénieurs,Public,0350097R,INSA RENNES,INST NAT SC APPLIQ RENNES,NaN,0350097R,INSA RENNES,INST NAT SC APPLIQ RENNES,...,12606,IRIS____0000000352380605,35238,Rennes,0605,352380605,Rennes,H,Campus de Beaulieu,Rennes
878,2019,Universités,Public,0290346U,U BREST,UNIVERSITE BREST,BRETAGNE OCCIDENTALE,0352076S,UFR SC VIE ENV,UFR SCIENCES VIE ENVIRONNEMENT,...,12606,IRIS____0000000352380605,35238,Rennes,0605,352380605,Rennes,H,Campus de Beaulieu,Rennes
1196,2015,Universités,Public,0350936C,U RENNES 1,UNIVERSITE RENNES 1,UNIVERSITE RENNES 1,0350075S,IG RENNES 1,INSTITUT DE GESTION RENNES 1,...,9919,IRIS____0000000352380203,35238,Rennes,0203,352380203,Rennes,H,Jean Macé,Rennes


In [37]:
iris_rennes_stats = iris_rennes.copy()

# Nombre d'établissements
iris_rennes_stats = iris_rennes_stats.merge(
    etabs_par_iris_rennes.groupby('LIB_IRIS').size().reset_index(name='nb_etabs'),
    on='LIB_IRIS', how='left'
).fillna({'nb_etabs':0})
iris_rennes_stats['nb_etabs'] = iris_rennes_stats['nb_etabs'].astype(int)

# Nombre total d'étudiants
iris_rennes_stats = iris_rennes_stats.merge(
    etabs_par_iris_rennes.groupby('LIB_IRIS')['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'].sum().reset_index(name='nb_etudiants'),
    on='LIB_IRIS', how='left'
).fillna({'nb_etudiants':0})
iris_rennes_stats['nb_etudiants'] = iris_rennes_stats['nb_etudiants'].astype(int)


In [38]:
iris_rennes_stats = iris_rennes_stats.to_crs(epsg=2154)  # crs projetée pour calculer surface en m²
iris_rennes_stats['area_km2'] = iris_rennes_stats.geometry.area / 1e6
iris_rennes_stats['etabs_per_km2'] = iris_rennes_stats['nb_etabs'] / iris_rennes_stats['area_km2']
iris_rennes_stats['students_per_km2'] = iris_rennes_stats['nb_etudiants'] / iris_rennes_stats['area_km2']


In [43]:
import plotly.express as px

# reconvertir en 4326
iris_plot = iris_rennes_stats.to_crs(epsg=4326)

fig = px.choropleth_mapbox(
    iris_plot,
    geojson=iris_plot.__geo_interface__,
    locations=iris_plot.index,
    color='students_per_km2',
    hover_name='LIB_IRIS',
    hover_data=['nb_etabs','area_km2', 'nb_etudiants'],
    mapbox_style="carto-positron",
    center={"lat":48.117, "lon":-1.677},
    zoom=12,
    opacity=0.6,
    color_continuous_scale="YlOrRd"
)

# ajouter les points OSM (facultatif)
fig.add_scattermapbox(
    lat=df_rennes['lat'],
    lon=df_rennes['lon'],
    mode='markers',
    marker=dict(size=6, color='blue'),
    text=etabs_par_iris_rennes["libellé de l'établissement"],
    name='Établissements OSM'
)

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0}, title="Densité étudiants sup. par IRIS - Rennes")
fig.show()


/var/folders/v0/_lflz19n6hv63x17d4_zjfjm0000gn/T/ipykernel_96163/3236287937.py:6: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [45]:
top10_iris = iris_rennes_stats.sort_values(by='students_per_km2', ascending=False).head(10)

fig = px.bar(
    top10_iris,
    x='LIB_IRIS',
    y='students_per_km2',
    hover_data=['nb_etudiants','area_km2','nb_etabs'],
    color='students_per_km2',
    color_continuous_scale='YlOrRd',
    title="Top 10 IRIS les plus denses en étudiants - Rennes"
)

fig.update_layout(xaxis_title="Quartier (IRIS)", yaxis_title="Étudiants / km²")
fig.show()


### Analyser les **quartiers vivants (nombre de resto, bars, et supermarchés)** à ? ###

In [ ]:
print("Partie Adam et Valentine")